In [23]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score, silhouette_score

In [2]:
data = pd.read_csv('data/cc_general.csv')
data.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [3]:
data.drop('CUST_ID', axis=1, inplace=True)

In [4]:
data.dropna(inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8636 entries, 0 to 8949
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   BALANCE                           8636 non-null   float64
 1   BALANCE_FREQUENCY                 8636 non-null   float64
 2   PURCHASES                         8636 non-null   float64
 3   ONEOFF_PURCHASES                  8636 non-null   float64
 4   INSTALLMENTS_PURCHASES            8636 non-null   float64
 5   CASH_ADVANCE                      8636 non-null   float64
 6   PURCHASES_FREQUENCY               8636 non-null   float64
 7   ONEOFF_PURCHASES_FREQUENCY        8636 non-null   float64
 8   PURCHASES_INSTALLMENTS_FREQUENCY  8636 non-null   float64
 9   CASH_ADVANCE_FREQUENCY            8636 non-null   float64
 10  CASH_ADVANCE_TRX                  8636 non-null   int64  
 11  PURCHASES_TRX                     8636 non-null   int64  
 12  CREDIT

In [6]:
# Find closely correlated features and remove them
num_corr = data.corr()
features = list(num_corr.columns)
print('original feature length: {}'.format(len(features)))

# Loops to check correlation under diagonal (where columns correlate with themselves)
for x in range(0, len(num_corr.columns), 1):
    col =  abs(num_corr.iloc[(x+1):, x]).sort_values(ascending=False)
    for y in range(len(col)):
        if col[y] > .75:
            if col.index[y] in features:
                features.remove(col.index[y])
print('new feature length: {}'.format(len(features)))

original feature length: 17
new feature length: 14


In [7]:
# replace data with updatad features list
data = data[features]
data.head()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,CASH_ADVANCE_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.40,95.40,0.000000,0.166667,0.000000,0.00,2,1000.0,201.802084,139.509787,0.000000,12
1,3202.467416,0.909091,0.00,0.00,6442.945483,0.000000,0.000000,0.25,0,7000.0,4103.032597,1072.340217,0.222222,12
2,2495.148862,1.000000,773.17,0.00,0.000000,1.000000,1.000000,0.00,12,7500.0,622.066742,627.284787,0.000000,12
4,817.714335,1.000000,16.00,0.00,0.000000,0.083333,0.083333,0.00,1,1200.0,678.334763,244.791237,0.000000,12
5,1809.828751,1.000000,1333.28,1333.28,0.000000,0.666667,0.000000,0.00,8,1800.0,1400.057770,2407.246035,0.000000,12


In [8]:
# Treat numeric columns with less than 10 unique values as categorical features and transform into dummy variables
for col in data.columns:    
    if len(data[col].unique()) < 10:
        dum = pd.get_dummies(data[col], prefix=col, drop_first=True)
        df = pd.concat([data, dum], axis=1)
        df.drop(col, axis=1, inplace=True)
df.head()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,CASH_ADVANCE_FREQUENCY,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE_7,TENURE_8,TENURE_9,TENURE_10,TENURE_11,TENURE_12
0,40.900749,0.818182,95.40,95.40,0.000000,0.166667,0.000000,0.00,2,1000.0,201.802084,139.509787,0.000000,0,0,0,0,0,1
1,3202.467416,0.909091,0.00,0.00,6442.945483,0.000000,0.000000,0.25,0,7000.0,4103.032597,1072.340217,0.222222,0,0,0,0,0,1
2,2495.148862,1.000000,773.17,0.00,0.000000,1.000000,1.000000,0.00,12,7500.0,622.066742,627.284787,0.000000,0,0,0,0,0,1
4,817.714335,1.000000,16.00,0.00,0.000000,0.083333,0.083333,0.00,1,1200.0,678.334763,244.791237,0.000000,0,0,0,0,0,1
5,1809.828751,1.000000,1333.28,1333.28,0.000000,0.666667,0.000000,0.00,8,1800.0,1400.057770,2407.246035,0.000000,0,0,0,0,0,1


In [9]:
# Scale data
scale = StandardScaler()
scaled = scale.fit_transform(df)

In [20]:
# Use DBSCAN for init exploratory clustering

# Function will rank best clustering parameters using adjusted_rand_score as the scoring metric
def best_dbscan_cluster(scaled):
    
    # Initialize values
    range_list = np.arange(1, 8, 1)
    best = 0
    score = {}
    
    # Loop through hyperparameters and save best scoring combinations
    for eps in range_list:
        
        for min_samp in range_list:
            dbscan = DBSCAN(eps=eps, min_samples=min_samp, n_jobs=8)
            clusters = dbscan.fit_predict(scaled)
            sil_score = silhouette_score(scaled, clusters, metric='euclidean')
                
            if sil_score > best:
                best = sil_score
                score['eps'] = eps
                score['min_samps'] = min_samp
                score['silhouette_score'] = sil_score
                score['clusters'] = len(np.unique(clusters))
                
    return score

In [21]:
score = best_dbscan_cluster(scaled)

In [22]:
score

{'eps': 6,
 'min_samps': 3,
 'silhouette_score': 0.4544023817402984,
 'clusters': 6}